In [23]:
file="/home/davide/Documents/operoniCCMEE029/list_of_operons_1404265"
with open(file) as f:
    lines = f.readlines()
operons = {}
for line in lines:
    #if line starts with number
    if line[0].isalpha():
        continue
    if line[0].isdigit():
        number = int(line.split("\t")[0])
        continue
    if number in operons:
        numbers = [word for word in line.split("\t") if word.isdigit()]
        assert len(numbers) == 2
        s,e = numbers[0], numbers[1]
        operons[number].append((s,e))
    else:
        s,e = line.split("\t")[4:6]
        operons[number] = [(s,e)]


In [24]:
print(operons)
#watch last key in dictionary and confirm all values between 1 and last key are present


{1: [('1', '241')], 2: [('570', '1055')], 3: [('1056', '3314'), ('3364', '4257'), ('4337', '5164'), ('5507', '6706'), ('6837', '7895'), ('7912', '8910'), ('8956', '9831'), ('9828', '10511')], 4: [('10587', '11294')], 5: [('11683', '12438')], 6: [('12746', '12940')], 7: [('13073', '13494'), ('13511', '13639')], 8: [('13907', '14698')], 9: [('14797', '15909')], 10: [('16356', '16727')], 11: [('17229', '18533')], 12: [('18686', '19579')], 13: [('19648', '21144')], 14: [('21371', '22354')], 15: [('22487', '25384')], 16: [('25468', '26118')], 17: [('26233', '26715')], 18: [('27246', '28382')], 19: [('28607', '29176')], 20: [('29290', '30546')], 21: [('30848', '32242')], 22: [('34449', '34847')], 23: [('35458', '35691'), ('35684', '35887'), ('35891', '36229'), ('36222', '37112'), ('37096', '37335'), ('37328', '38005'), ('38002', '38916'), ('38909', '39388')], 24: [('39526', '39831'), ('39863', '40054')], 25: [('40142', '40411'), ('40445', '40672')], 26: [('40940', '41137'), ('41188', '41544'

In [15]:
last_key = max(operons.keys())
missing_keys = []

for i in range(1, last_key + 1):
    if i not in operons:
        missing_keys.append(i)

if missing_keys:
    print(f"Missing keys: {missing_keys}")
else:
    print("All keys between 1 and the last key are present.")

All keys between 1 and the last key are present.


In [42]:
gbk="annotati_Refseq/Chroococcidiopsis_sp._CCMEE_29_(cyanobacteria)_GCF_023558375.gbff"
from Bio import SeqIO
operons_locus_tags = {}
for record in SeqIO.parse(gbk, "genbank"):
    #iterate over CDS features
    for feature in record.features:
        if feature.type == "CDS":
            s,e=feature.location.start, feature.location.end
            s=int(s)+1
            e=int(e)
            locus_tag = feature.qualifiers['locus_tag'][0]
            pid= feature.qualifiers.get('protein_id', [''])[0]
            #check if CDS is in an operon
            for key in operons:
                for start, end in operons[key]:
                    if s==int(start) and e==int(end):
                        if key in operons_locus_tags:
                            operons_locus_tags[key].append((locus_tag, pid))
                        else:
                            operons_locus_tags[key] = [(locus_tag, pid)]
            

In [43]:
print(operons_locus_tags)

{1: [('LAU37_RS00005', 'WP_275983370.1')], 2: [('LAU37_RS00010', 'WP_250123595.1')], 3: [('LAU37_RS00015', 'WP_250123596.1'), ('LAU37_RS00020', 'WP_250123597.1'), ('LAU37_RS00025', 'WP_250123598.1'), ('LAU37_RS00035', 'WP_250123600.1'), ('LAU37_RS00040', 'WP_250123601.1'), ('LAU37_RS00045', 'WP_250123602.1'), ('LAU37_RS00050', 'WP_250123603.1')], 4: [('LAU37_RS00055', 'WP_250123604.1')], 5: [('LAU37_RS00060', 'WP_250123605.1')], 6: [('LAU37_RS00065', 'WP_250123606.1')], 7: [('LAU37_RS00075', '')], 8: [('LAU37_RS00080', 'WP_250123608.1')], 9: [('LAU37_RS00085', 'WP_250123609.1')], 10: [('LAU37_RS00090', 'WP_250123610.1')], 11: [('LAU37_RS00095', 'WP_250123611.1')], 12: [('LAU37_RS00100', 'WP_250123612.1')], 13: [('LAU37_RS00105', 'WP_250123613.1')], 14: [('LAU37_RS00110', 'WP_250123614.1')], 15: [('LAU37_RS00115', 'WP_250123615.1')], 16: [('LAU37_RS00120', 'WP_250123616.1')], 17: [('LAU37_RS00125', 'WP_250123617.1')], 18: [('LAU37_RS00130', 'WP_250123618.1')], 19: [('LAU37_RS00135', 'WP

In [ ]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
def sequenze_intergenichetutteOperoni(input, output, max_len=300,c=0,operoni):
    inizi=[oprone[0][0] for oprone in operoni.values()]
    intergeniche = []  # Lista per memorizzare le sequenze intergeniche
    genoma = SeqIO.parse(input, "genbank")
    # Loop attraverso le features nel file GenBank
    for sequenze in genoma:
        # sequenze sta per sequenzefold perchè il file genbank potrebbe non contenere un solo cromosoma circolare
        cds_list_plus = []  # Lista per memorizzare le informazioni sui geni in direzione +
        cds_list_minus = []  # Lista per memorizzare le informazioni sui geni in direzione -
        for feature in sequenze.features:
            # Controlla se la feature è un CDS
            if feature.type == "CDS":
                mystart = feature.location.start  # Ottieni l'inizio del CDS
                myend = feature.location.end  # Ottieni la fine del CDS
                mystrand = '-' if feature.location.strand == -1 else '+'
                mystart, myend = int(mystart), int(myend)
                # Check the strand of the CDS and add the info to the appropriate list
                if mystrand == '-':
                    cds_list_minus.append((mystart, myend, feature.qualifiers["locus_tag"][0],
                                           feature.qualifiers["product"][0],
                                           feature.qualifiers.get("gene", [' '])[0]))
                else:
                    cds_list_plus.append((mystart, myend, feature.qualifiers["locus_tag"][0],
                                          feature.qualifiers["product"][0],
                                          feature.qualifiers.get("gene", [' '])[0]))

        # Loop attraverso i geni in direzione +
        for i, n in enumerate(cds_list_plus):
            last_end = 0 if i == 0 else cds_list_plus[i - 1][1]
            this_start = n[0]  # Ottieni l'inizio del gene corrente
            intergene_seq = sequenze.seq[last_end:this_start+c]  # Ottieni la sequenza intergenica
            if len(intergene_seq) >= max_len:  # Se la sequenza intergenica è più lunga di max_len, prendi solo le ultime max_len basi(default 300)
                intergene_seq = sequenze.seq[this_start - max_len:this_start+c]
                last_end = this_start - max_len  # Aggiorna quello che sarà l'inizio della sequenza intergenica
            # Aggiungi la sequenza intergenica alla lista come SeqRecord
            description = f"product = {cds_list_plus[i][3]}, gene = {cds_list_plus[i][4]} {last_end + 1}-{this_start+c} +"
            if len(intergene_seq) == 0:
                continue
            intergeniche.append(
                SeqRecord(
                    intergene_seq,
                    id=n[2],  # id del gene(locus_tag)
                    description=description
                )
            )
        # Loop attraverso i geni in direzione -
        for i, n in enumerate(cds_list_minus):
            next_start = len(sequenze.seq) if i + 1 == len(cds_list_minus) else cds_list_minus[i + 1][0]  # Ottieni l'inizio del prossimo gene(gene precedente per la direzione -)
            this_end = n[1]  # Ottieni la fine del gene corrente(inizio del gene per la direzione -)
            intergene_seq = sequenze.seq[this_end-c:next_start]  # Ottieni la sequenza intergenica
            if len(intergene_seq) >= max_len:
                # Se la sequenza intergenica è più lunga di max_len, prendi solo le prime max_len basi(default 300)
                intergene_seq = sequenze.seq[this_end-c:this_end + max_len] if this_end + max_len < len(
                    sequenze.seq) else sequenze.seq[this_end-c:]
                next_start = this_end + max_len
            intergene_seq = intergene_seq.reverse_complement()  # Complemento inverso della sequenza intergenica
            description = f"product = {cds_list_minus[i][3]}, gene = {cds_list_minus[i][4]} {this_end-c}-{next_start + 1} -"
            # Aggiungi la sequenza intergenica alla lista come SeqRecord
            if len(intergene_seq) == 0:
                continue
            intergeniche.append(
                SeqRecord(
                    intergene_seq,
                    id=n[2],  # id del gene(locus_tag)
                    description=description
                )
            )

    # Scrivi le sequenze intergeniche nel file di output
    SeqIO.write(intergeniche, output, "fasta")